In [1]:
import tensorflow as tf
import numpy as np
from hourglass_tiny import HourglassModel
import configparser
import os
import pandas as pd
from keras.preprocessing import image
import scipy.io
import matplotlib.pyplot as plt
from skimage.transform import resize

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [2]:
# read the all names of files that contains person
# get the path of masks and the path of data
person_names_mat = os.listdir('VOC2010/masks')
person_names_jpg = []
for i in range(len(person_names_mat)):
    temp = person_names_mat[i][:-4]+'.jpg'
    person_names_jpg.append(temp)
mat_path = []
for i in range(len(person_names_mat)):
    path = os.path.join('VOC2010/masks',person_names_mat[i])
    mat_path.append(path)
jpg_path = []
for i in range(len(person_names_jpg)):
    path = os.path.join('VOC2010/JPEGImages',person_names_jpg[i])
    jpg_path.append(path)
data_table = pd.DataFrame(jpg_path,columns=['jpg_path'])
mat_path_df = pd.DataFrame(mat_path,columns=['mat_path'])
data_table = pd.concat([data_table,mat_path_df],axis=1)
data_table.head()

,jpg_path,mat_path
0,VOC2010/JPEGImages\2008_000008.jpg,VOC2010/masks\2008_000008.mat
1,VOC2010/JPEGImages\2008_000023.jpg,VOC2010/masks\2008_000023.mat
2,VOC2010/JPEGImages\2008_000036.jpg,VOC2010/masks\2008_000036.mat
3,VOC2010/JPEGImages\2008_000041.jpg,VOC2010/masks\2008_000041.mat
4,VOC2010/JPEGImages\2008_000096.jpg,VOC2010/masks\2008_000096.mat


In [3]:
def squre_crop(img,mask):
    # overlap img and mask then crop
    overlap = np.zeros([img.shape[0],img.shape[1],img.shape[2]+1])
    overlap[:,:,0:3] = img
    overlap[:,:,3] = mask
    # random crop
    height = overlap.shape[0]
    width = overlap.shape[1]
    if height%2 != 0:
        height = height - 1
    if width%2 != 0:
        width = width - 1
    if width>height:
        range1 = [0,height]
        range2 = [width-height,width]
        range3 = [int(width/2-height/2),int(width/2+height/2)]
        crop1 = overlap[:,range1[0]:range1[1],:]
        crop2 = overlap[:,range2[0]:range2[1],:]
        crop3 = overlap[:,range3[0]:range3[1],:]
    else:
        range1 = [0,width]
        range2 = [height-width,height]
        range3 = [int(height/2-width/2),int(height/2+width/2)]
        crop1 = overlap[range1[0]:range1[1],:,:]
        crop2 = overlap[range2[0]:range2[1],:,:]
        crop3 = overlap[range3[0]:range3[1],:,:]
    return crop1,crop2,crop3

In [4]:
# read and preprosess the data
# read img
Data = []
for i in range(30):
    test_img = image.load_img(data_table['jpg_path'][i])
    test_img = image.img_to_array(test_img)
    # read mask
    test_mask = scipy.io.loadmat(data_table['mat_path'][i])['single_map']
    crop1,crop2,crop3 = squre_crop(test_img,test_mask)
    re_crop1 = resize(crop1,output_shape=[256,256])
    re_crop2 = resize(crop2,output_shape=[256,256])
    re_crop3 = resize(crop3,output_shape=[256,256])
    Data.append(re_crop1)
    Data.append(re_crop2)
    Data.append(re_crop3)

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
# split the data into train and validation
train_data = Data[0:70]
val_data = Data[70:90]

In [6]:
def get_next_batch(data):
    length = len(data)
    idx = np.random.randint(0,length)
    img = data[idx][:,:,0:3]
    mask = data[idx][:,:,3]
    return img,mask

## ---test---

In [31]:
img,mask = get_next_batch(train_data)
plt.figure()
plt.imshow(img/255)
plt.show()
plt.figure()
plt.imshow(mask)
plt.show()

## ---test end---

In [8]:
def augmentor_with_keras(img,mask):
    temp = np.zeros([256,256,4])
    temp[:,:,0:3] = img
    temp[:,:,3]= mask
    after_rot = image.random_rotation(temp,rg=20,
                                      row_axis=0,col_axis=1,channel_axis=2,fill_mode='constant')
    after_zoom = image.random_zoom(after_rot,zoom_range=[0.7,1.3],
                                   row_axis=0,col_axis=1,channel_axis=2,fill_mode='constant')
    # random_shift
    return after_zoom[:,:,0:3],after_zoom[:,:,3]

## ---test---

In [35]:
aug_img,aug_mask = augmentor_with_keras(img,mask)
plt.figure()
plt.imshow(aug_img/255)
plt.show()
plt.figure()
plt.imshow(aug_mask)
plt.show()

## ---test end---

In [ ]:
def my_scale(aug_mask):
    for w in range(256):
        for h in range(256):
            if aug_mask[w,h]%1 != 0:
                aug_mask[w,h] = 0
    scale_map = np.zeros([64,64])
    for i in range(64):
        for j in range(64):
            scale_map[i,j] = aug_mask[i*4+2,j*4+1]
    return scale_map


In [ ]:
def transfer_mask_to_gtmap(mask):
    mask = my_scale(mask)
    map = np.zeros([64,64,15])
    for i in range(15):
        for n in range(64):
            for m in range(64):
                if mask[n][m] == i:
                    map[n,m,i] = 1.0
    # transform to gtMap
    gtmap = np.zeros([4,64,64,15])
    for i in range(4):
        gtmap[i] = map
    return gtmap

def get_augmentor_batch(img,mask,aug_level):
    gtmap = transfer_mask_to_gtmap(mask)
    x_batch = np.zeros([aug_level,256,256,3])
    y_batch = np.zeros([aug_level,4,64,64,15])
    x_batch[0]=img
    y_batch[0]=gtmap
    for i in range(aug_level-1):
        aug_img,aug_mask = augmentor_with_keras(img,mask)
        gtmap = transfer_mask_to_gtmap(aug_mask)
        x_batch[i+1] = aug_img
        y_batch[i+1] = gtmap
    return x_batch,y_batch

In [ ]:
def cal_acc(pred,mask):
    length = pred.shape[0]
    wideth = pred.shape[1]
    help_arr = np.zeros([length,wideth])
    for i in range(length):
        for j in range(wideth):
            if pred[i,j] == mask[i,j]:
                help_arr[i,j] = 1
    acc = np.mean(help_arr)
    return acc

In [ ]:
def cal_iou(pred,mask):
    length = pred.shape[0]
    wideth = pred.shape[1]
    # find the pixel that at the same time in two map
    p_in_two = 0
    for i in range(length):
        for j in range(wideth):
            if (pred[i,j] != 0) and (mask[i,j] != 0):
                p_in_two = p_in_two + 1
    # find the pixel that not equal to zero
    p_all = 0
    for i in range(length):
        for j in range(wideth):
            if (pred[i,j] != 0) or (mask[i,j] != 0):
                p_all = p_all + 1
    if p_all != 0:
        iou = p_in_two/p_all
    else:
        iou = 0
    return iou

In [ ]:
def process_config(conf_file):
	"""
	"""
	params = {}
	config = configparser.ConfigParser()
	config.read(conf_file)
	for section in config.sections():
		if section == 'DataSetHG':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Network':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Train':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Validation':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Saver':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
	return params
print('--Parsing Config File')
params = process_config('config.cfg')

In [ ]:
model = HourglassModel(nFeat=params['nfeats'], nStack=params['nstacks'], 
                       nModules=params['nmodules'], 
                       nLow=params['nlow'], outputDim=params['num_joints'], 
                       batch_size=params['batch_size'], attention = params['mcam'],
                       training=True, drop_rate= params['dropout_rate'], 
                       lear_rate=params['learning_rate'], 
                       decay=params['learning_rate_decay'], 
                       decay_step=params['decay_step'], 
                       name=params['name'], 
                       logdir_train=params['log_dir_train'], 
                       logdir_test=params['log_dir_test'], 
                       tiny= params['tiny'], w_loss=params['weighted_loss'] , 
                       joints= params['joint_list'],modif=False)
model.generate_model()

In [ ]:
# output: (None,8,64,64,15)
output = model.get_output()
loss = model.loss
out = output[0,7]
pred_seg = tf.argmax(out,axis=2)
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

In [ ]:
saver = tf.train.Saver()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
# train
iteration = []
acc_list = []
iou_list = []
for i in range(10):
    x_b,y_b = get_next_batch(train_data,train_label)
    x,y = get_augmentor_batch(x_b,y_b,3)
    sess.run(train_step,feed_dict={model.img:x, model.gtMaps:y})

    x_for_eval = np.expand_dims(x_b,axis=0)
    pred = sess.run(pred_seg,feed_dict={model.img:x_for_eval})
    acc = cal_acc(pred,y_b)
    iou = cal_iou(pred,y_b)
    print('acc:%f',acc,'  ','iou:%f',iou)
    iteration.append(i)
    acc_list.append(acc)
    iou_list.append(iou)

In [ ]:
# save iou and acc
eval_info = pd.DataFrame([iteration,acc_list,iou_list],columns=['iter','accracy','iou'])
eval_info.to_csv('training_info_with_pascal.csv')

In [ ]:
# save the network
path = "my_net_with_pascal"
is_exist = os.path.exists(path)
if not is_exist:
    os.makedirs(path)
full_path = os.path.join(path,'save_net.ckpt')
save_path = saver.save(sess,full_path)